In [ ]:
import gradelib
import gradelib.steps as gs

gb = gradelib.combine_gradebooks([
    gradelib.io.gradescope.read("./gradescope.csv"),
    gradelib.io.canvas.read("./canvas.csv")
])

gb = gb.apply([
    # merge homeworks and homework autograded parts
    gs.CombineAssignments(lambda s: s.split('-')[0]),
])

HOMEWORKS = LazyAssignments(lambda asmts: asmts.starting_with("homework"))
LABS = LazyAssignments(lambda asmts: asmts.starting_with("lab"))

EXCEPTIONS = [
    gs.Exception(
        "Justin Eldridge",
        forgive_lates=["homework 01", "homework 03"],
        drop=["homework 04"],
        replace=[("homework 05", "midterm 01")],
        adjust=[
            ("homework 06", Points(32)),
            ("midterm 02", Percentage(.85))
        ]
    )
]

POLICIES = [
    gs.PenalizeLates(
        forgive=3,
        within=lambda asmts: asmts.starting_with("homework"),
        penalty=Percentage(.75)
    ),
    
    gs.DropLowest(
        2,
        within=HOMEWORKS + LABS,
    ),
    
    gs.Redemption("midterm 01", "redemption midterm 01"),
    gs.Redemption("midterm 02", "redemption midterm 02") 
]

pipeline = [
    *EXCEPTIONS,
    *POLICIES
]

fgb = gb.grade(
    pipeline=pipeline,
)

fgb.opts.allow_extra_credit = True

fgb.groups = [
    (HOMEWORKS, 0.5),
    (LABS, 0.3),
    ('midterm 01', 0.25),
    ('midterm 02', 0.25),
    ('extra credit', .03)
]

fgb.scale = gradelib.io.read_scale()

scale = gradelib.scale.find_robust_scale(fgb)
gradelib.io.write_scale(scale, overwrite=True)

fgb.overall_scores
fgb.letter_grades
fgb.summarize()

fgb.summarize("someone")

In [ ]:
MutableGradebook

# these should all mutate the gradebook

.add_assignment()
.restrict_to_assignments()
.remove_assignment()
.combine_assignments()
.restrict_to_students()
.add_note()

# these should produce copies
.apply()
.finalize()
.grade() = .apply() + .finalize()

FinalizedGradebook